In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm

In [20]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/



**1.1** **Get the list of animes**

With the for loop I get the url of all the pages (50 per page except the last one which has less,383 pag).
We 'send' a request to the page, to collect the ursl.
We read the html and take the href attributes of the tag.
The command tqdm is useful because it shows the progress status.


In [25]:
urls=[]
for i in tqdm(range(0,20000,50)):
  url='https://myanimelist.net/topanime.php?limit='+str(i)
  
  soup= BeautifulSoup(requests.get(url).text,'html.parser')

  for tag in soup.find_all('tr'):
    links=tag.find_all('a')
    for l in links:
      if type(l.get('id'))== str and len(l.contents[0]) >1:
        urls.append(l.get('href'))



100%|██████████| 400/400 [02:47<00:00,  2.38it/s]


In [ ]:
len(urls)
 

We need to save it as text


In [ ]:
with open('urls.txt', 'w', encoding='utf-8') as f: #open the file con write, chiude da solo in automatico, lo trovo in file sample data
    for line in urls:
        f.write(line)
        f.write('\n')

In [ ]:
#!ls pages/3pag & just a check, this is used to see what there is inside 3pag


**1.2** **Crawl animes**

Create the folders. Put all of them under "pages". Each folder has a name that refers to the number of the page from which the links it contains come from.

In [21]:
import os
for page in tqdm(range(1, 384)):
    folder = "page"+str(page)
    path = "/content/drive/MyDrive/Anime_folder"+folder
    os.mkdir(path)

100%|██████████| 383/383 [00:01<00:00, 328.29it/s]


In [16]:
!rm -rf pages
   #to delete folders that are not needed.

Try for the first 150:ok.
Try to collect them in group of 3.


In [28]:
for page in tqdm(range(0, 3)):  # page 1 --> 383
    folder = "/content/drive/MyDrive/Anime_folderpage"+str(page+1)
    update_page = 50*page
    for i in range(0,50):   # 1 -> 50
        url = f'{urls[update_page+i]}'
        response = requests.get(url)   
        filename = r""+folder+"/anime_"+str(update_page+i+1)+".html"
        with open(filename,'w', encoding='utf-8') as f:
            f.write(response.text)

100%|██████████| 3/3 [01:01<00:00, 20.50s/it]


In [177]:
for page in tqdm(range(4, 8)):  # page 1 --> 383
    folder = "/content/drive/MyDrive/Anime_folderpage"+str(page+1)
    update_page = 50*page
    for i in range(0,50):   # 1 -> 50
        url = f'{urls[update_page+i]}'
        response = requests.get(url)   
        filename = r""+folder+"/anime_"+str(update_page+i+1)+".html"
        with open(filename,'w', encoding='utf-8') as f:
            f.write(response.text)

100%|██████████| 4/4 [01:31<00:00, 22.99s/it]


**1.3**
Parsing downloaded pages.


In [ ]:
'''col = ['animeTitle', 'animeType', 'animeNumEpisode', 'releaseDate', 'endDate', 'animeNumMembers', 'animeScore', \
           'animeUsers', 'animeRank', 'animePopularity', 'animeDescription', 'animeRelated', 'animeCharacters', \
           'animeVoices', 'animeStaff']
           '''

In [29]:
def AnimeTitle(html):
  with open(html,'r') as f:
    soup= BeautifulSoup(f)
    animeTitle=soup.find("h1", attrs = {"class": "title-name h1_bold_none"}).string
    return(animeTitle)


In [30]:
AnimeTitle('/content/drive/MyDrive/Anime_folderpage1/anime_10.html')

'Gintama: The Final'

In [49]:
animeScore('/content/drive/MyDrive/Anime_folderpage1/anime_13.html')

8.97

In [40]:
def animeScore(html):
  with open(html,'r') as f:
    soup= BeautifulSoup(f)
    for i in range(10):
      score=soup.find_all('div' ,attrs = {"class": "score-label score-"+str(i)})
      for j in score:
        return float(j.text)

In [54]:
def AnimeRank(html):
  with open(html,'r') as f:
    soup= BeautifulSoup(f)
    rank=int(str(soup.find(class_="numbers ranked").text).split('#')[-1])  
    return(rank)


In [55]:
AnimeRank('/content/drive/MyDrive/Anime_folderpage1/anime_13.html')


13

In [89]:
import re

In [93]:
def animeNumberofepisode(html_string):
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A=soup.find(text=re.compile('Episodes:')).parent.parent.text.split()[-1]      
    print(A)

In [92]:
animeNumberofepisode('/content/drive/MyDrive/Anime_folderpage1/anime_1.html')


64


In [58]:
def AnimePopularity(html):
  with open(html,'r') as f:
    soup= BeautifulSoup(f)
    rank=int(str(soup.find(class_="numbers popularity").text).split('#')[-1])  
    return(rank)

In [59]:
AnimePopularity('/content/drive/MyDrive/Anime_folderpage1/anime_13.html')

23

In [70]:
def animeDescription(html_string): #I think that this function works well.
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    A=soup.find(itemprop="description")
    return A.get_text()

In [71]:
animeDescription('/content/drive/MyDrive/Anime_folderpage1/anime_10.html')

'New Gintama movie.'

In [75]:
def animeUsers(html_string): #I think that this function works well.
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    A=soup.find(itemprop="ratingCount")
    return int(A.get_text())

In [85]:
animeUsers('/content/drive/MyDrive/Anime_folderpage1/anime_13.html')

1208990

In [97]:
def animeType(html_string):
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A=soup.find(text=re.compile('Type:')).parent.parent.text.split()[-1]      
    print(A)

In [99]:
animeType('/content/drive/MyDrive/Anime_folderpage1/anime_11.html')

TV


 turn a list of strings in datetime format.
   

In [173]:
def fai_date(a: list):
    
    string = ''
    for i in a:
        string += i
        if i != a[-1]:
            string += ' '
    string = string.replace(',', '')
    date = datetime.strptime(string, '%b %d %Y')
    return date

In [174]:
def animeRelDate(html_string):
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A= fai_date(soup.find(text=re.compile('Aired:'), class_="dark_text").parent.text.split()[1:4]  )   
    print(A)

In [175]:
animeRelDate('/content/drive/MyDrive/Anime_folderpage1/anime_11.html')

2017-01-09 00:00:00


In [176]:
def animeEndDate(html_string):
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A= fai_date(soup.find(text=re.compile('Aired:'), class_="dark_text").parent.text.split()[5:8]   )  
    print(A)

In [121]:
animeEndDate('/content/drive/MyDrive/Anime_folderpage1/anime_11.html')

['Mar', '27,', '2017']


In [135]:
def animeRelated(html_string):  
  animeRelated = []
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    related = soup.find_all("table", {"class":"anime_detail_related_anime"})
    for i in related:
      links = i.find_all('a')
      for link in links:        
          animeRelated.append(f'{link.contents[0]}')

  return animeRelated
     

In [136]:
 animeRelated('/content/drive/MyDrive/Anime_folderpage1/anime_11.html')

['Gintama', 'Gintama°', 'Gintama.: Porori-hen']

In [147]:
def animeCharacter(html_string):
  personaggi=[]
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A= soup.find_all(class_="h3_characters_voice_actors")    
    for a in A:
      personaggi.append(a.text)
    return(personaggi) 

In [148]:
animeCharacter('/content/drive/MyDrive/Anime_folderpage1/anime_11.html')

['Sakata, Gintoki',
 'Kagura',
 'Katsura, Kotarou',
 'Takasugi, Shinsuke',
 'Shimura, Shinpachi',
 'Kamui',
 'Elizabeth',
 'Imai, Nobume',
 'Sadaharu',
 'Sakamoto, Tatsuma']

In [162]:
def animeVoices(html_string):
  personaggi=[]
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A= soup.find_all(class_="va-t ar pl4 pr4")   
    for a in A:
      personaggi.append(a.text)

      
    return(personaggi) 
   

In [163]:
animeVoices('/content/drive/MyDrive/Anime_folderpage1/anime_11.html')

['\nSugita, Tomokazu\nJapanese\n',
 '\nKugimiya, Rie\nJapanese\n',
 '\nIshida, Akira\nJapanese\n',
 '\nKoyasu, Takehito\nJapanese\n',
 '\nSakaguchi, Daisuke\nJapanese\n',
 '\nHino, Satoshi\nJapanese\n',
 '\nHirano, Aya\nJapanese\n',
 '\nTakahashi, Mikako\nJapanese\n',
 '\nMiki, Shinichiro\nJapanese\n']

In [171]:
def createstaff(ll):
    """
    puts all relevant text in a list after cleaning it up a little
    """
    l = []
    for i in ll:
        if i['href'].startswith('https://myanimelist.net/people/') and i['href'] not in l:
            if i.text != '\n\n':
                j = str(i.text).replace('\n', '')
                j = re.sub(' +', ' ', j)
            if j[0] == ' ':
                j = j[1:]
            if j[-1] == ' ':
                j = j[:-1]
            l.append(j)
    return l      





def animeStaff(html_string):
   with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A=  createstaff(soup.find_all('a', href=True))     
    print(A)
  
  

In [172]:
animeStaff('/content/drive/MyDrive/Anime_folderpage1/anime_11.html')

['Sugita, Tomokazu', 'Sugita, Tomokazu', 'Kugimiya, Rie', 'Kugimiya, Rie', 'Ishida, Akira', 'Ishida, Akira', 'Koyasu, Takehito', 'Koyasu, Takehito', 'Sakaguchi, Daisuke', 'Sakaguchi, Daisuke', 'Hino, Satoshi', 'Hino, Satoshi', 'Hirano, Aya', 'Hirano, Aya', 'Takahashi, Mikako', 'Takahashi, Mikako', 'Miki, Shinichiro', 'Miki, Shinichiro', 'Miki, Shinichiro', 'Fujita, Youichi', 'Fujita, Youichi', 'Miyawaki, Chizuru', 'Miyawaki, Chizuru', 'Takamatsu, Shinji', 'Takamatsu, Shinji', 'Kobayashi, Katsuyoshi']
